In [33]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Data Augmentation(Image transformations) 을 하기 위해 필요한 모듈

import scipy
import keras
from tensorflow.keras.utils import plot_model
import keras_tuner as kt
import shutil

In [34]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2021-11-28 01:24:02.220512: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 01:24:02.222532: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5039637373826999566
 xla_global_id: -1]

In [99]:
# Data Loading
# 1. 기본 뿌리 디렉터리와 검증데이터 경로를 가져온다
#base_path = os.path.abspath('/media/chicago/Samsung_T5/Bigdata_Project/training_2000_data')
#base_path = os.path.abspath('/media/chicago/Samsung_T5/Bigdata_Project/training_balanced_data')
#val_path = os.path.abspath('/media/chicago/Samsung_T5/Bigdata_Project/real_data')
#base_path = os.path.abspath('/media/chicago/Samsung_T5/Bigdata_Project/first_checking_data')

#base_path = os.path.abspath('/home/chicago/Desktop/hopefully_main_data_with_Kaggle')
base_path = os.path.abspath('/home/chicago/semi_training_balanced_data')
#val_path = os.path.abspath('/home/chicago/Desktop/hopefully_main_data')

#base_path = os.path.abspath('/media/chicago/Samsung_T5/Bigdata_Project/first_checking_data')
print(base_path)

# 2. 데이터 경로를 지정을 해준다.
no_pothole_dir = os.path.join(base_path, '0. stone')
pothole_dir = os.path.join(base_path, '1. pothole')
filled_dir = os.path.join(base_path, '2. filled')

# 3. 데이터 셋의 개수를 카운팅한다.
num_no_pothole = len(os.listdir(no_pothole_dir))
num_pothole_dir = len(os.listdir(pothole_dir))
num_filled_dir = len(os.listdir(filled_dir))

total_data = num_no_pothole + num_pothole_dir + num_filled_dir

print(f'정상도로 : {num_no_pothole} 개 , 팟홀도로 : {num_pothole_dir}, 보수도로 : {num_filled_dir}')

print(f'Total Data 개수 : {total_data}')

# Train set 과 Validation set 을 나누며, Data Augmentation 작업 또한 추가시킨다.
#11/12 여기 배치 정규화 추가하는 코드 찾으면 넣자

train_image_generator = ImageDataGenerator(rescale=1./255,
      rotation_range=6,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.15)
'''

# without augmentation
train_image_generator = ImageDataGenerator(rescale=1./255,
      validation_split=0.15)
'''

validation_image_generator = ImageDataGenerator(rescale=1./255,
                                                horizontal_flip=True,
                                                validation_split=0.15)

train_batch_size = 128
val_batch_size = 128

epochs = 10
IMG_HEIGHT = 320
IMG_WIDTH = 320

#train_batch_size = 64
#val_batch_size = 64
'''
train_data_gen = train_image_generator.flow_from_directory(batch_size=train_batch_size,
                                                           directory=base_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=val_batch_size,
                                                           directory='/media/chicago/Samsung_T5/Bigdata_Project/real_data',
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')
'''
#'''Normal Training
train_data_gen = train_image_generator.flow_from_directory(batch_size=train_batch_size,
                                                           directory=base_path,
                                                           seed = 21,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='training',
                                                           class_mode='categorical')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=val_batch_size,
                                                           directory=base_path,
                                                           seed = 21,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='validation',
                                                           class_mode='categorical')

'''
test_image_generator = ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(batch_size=1,
                                                        directory='/media/chicago/Samsung_T5/Bigdata_Project/real_data',
                                                        target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                        class_mode='categorical')

'''

train_num = train_data_gen.samples # training set 이 몇개인지
val_num = val_data_gen.samples   # validation set 이 몇개인지

/home/chicago/semi_training_balanced_data
정상도로 : 6813 개 , 팟홀도로 : 6815, 보수도로 : 6631
Total Data 개수 : 20259
Found 17222 images belonging to 3 classes.
Found 3037 images belonging to 3 classes.


In [95]:
### Chill

feature_extractor = tf.keras.models.load_model('/home/chicago/Downloads/model_11 (1).h5')

In [82]:
### Chill
base_model = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=0.35,
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        pooling=None
      )
base_model.trainable = True

inputs = keras.Input(shape=(320, 320, 3))
x = base_model(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units=320,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=128,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)
'''
x = tf.keras.layers.Dense(units=96,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)
'''
x = tf.keras.layers.Dense(units=32,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

new_model = keras.Model(inputs=inputs, outputs=outputs)

new_model.summary()

plot_model(new_model, show_shapes=True)

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_32 (InputLayer)       [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_15  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_80 (Dense)            (None, 320)               409920    
                                                                 
 batch_normalization_65 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                          

In [83]:
### Chill
# first index is about feature extractor (1: feature extractor 얼리는거 3,4 는 상위층 !)
new_model.layers[1].set_weights(feature_extractor.layers[1].get_weights())
new_model.layers[3].set_weights(feature_extractor.layers[3].get_weights())
new_model.layers[4].set_weights(feature_extractor.layers[4].get_weights())
# 특징추출기에 해당하는 CNN 을 freezing 시킨다.
new_model.layers[1].trainable = True
new_model.layers[3].trainable = True
new_model.layers[4].trainable = True

new_model.summary()

#opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.0, nesterov=False)
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, centered=True)
#opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
#opt = tf.keras.optimizers.Nadam(learning_rate=0.00166, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
new_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics = ['accuracy'])
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/chicago/model_15.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

history = new_model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=10,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks])

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_32 (InputLayer)       [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_15  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_80 (Dense)            (None, 320)               409920    
                                                                 
 batch_normalization_65 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                          

/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
134/134 [==============================] - 1114s 8s/step - loss: 0.4046 - accuracy: 0.8399 - val_loss: 1.0226 - val_accuracy: 0.7500
Epoch 2/10
134/134 [==============================] - 1100s 8s/step - loss: 0.3512 - accuracy: 0.8631 - val_loss: 0.8220 - val_accuracy: 0.7425
Epoch 3/10
134/134 [==============================] - 1100s 8s/step - loss: 0.3267 - accuracy: 0.8716 - val_loss: 0.8255 - val_accuracy: 0.7204
Epoch 4/10
134/134 [==============================] - 1101s 8s/step - loss: 0.3145 - accuracy: 0.8751 - val_loss: 0.9371 - val_accuracy: 0.7052
Epoch 5/10
134/134 [==============================] - 1100s 8s/step - loss: 0.3038 - accuracy: 0.8808 - val_loss: 0.8821 - val_accuracy: 0.7031
Epoch 6/10
134/134 [==============================] - 1109s 8s/step - loss: 0.2952 - accuracy: 0.8856 - val_loss: 0.9854 - val_accuracy: 0.6596
Epoch 7/10
 73/134 [===============>..............] - ETA: 7:16 - loss: 0.2761 - accuracy: 0.8937

KeyboardInterrupt: 

In [86]:
history.history

{'loss': [0.4021032452583313,
  0.3474816083908081,
  0.3298422396183014,
  0.31699714064598083,
  0.3088819086551666,
  0.29809948801994324,
  0.2902112305164337,
  0.28028208017349243,
  0.2746274471282959,
  0.26395660638809204,
  0.25945672392845154,
  0.25344493985176086,
  0.24265584349632263,
  0.23330806195735931,
  0.23250927031040192,
  0.22402341663837433,
  0.22053013741970062,
  0.22061190009117126,
  0.20931082963943481,
  0.2103223204612732],
 'accuracy': [0.8388323187828064,
  0.8623493909835815,
  0.8711243867874146,
  0.8735813498497009,
  0.8807184100151062,
  0.8831753730773926,
  0.8858664035797119,
  0.8915408849716187,
  0.8935298919677734,
  0.896981418132782,
  0.898034393787384,
  0.9023634195327759,
  0.9086813926696777,
  0.9089154005050659,
  0.9130104184150696,
  0.9135369062423706,
  0.9145899415016174,
  0.9160524010658264,
  0.9192114472389221,
  0.9202058911323547],
 'val_loss': [1.3224236965179443,
  1.1335694789886475,
  1.5108853578567505,
  0.92605

In [100]:
from tensorflow.python.keras import regularizers

### Chill
base_model1 = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=0.35,
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        pooling=None
      )
base_model1.trainable = True

inputs = keras.Input(shape=(320, 320, 3))
x = base_model1(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units=320,activation=None, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.l2(1e-4))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=128,activation=None, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.l2(1e-4))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=96,activation=None,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.l2(1e-4))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)

x = tf.keras.layers.Dense(units=32,activation=None,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.l2(1e-4))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(activation="elu")(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

new_model1 = keras.Model(inputs=inputs, outputs=outputs)

new_model1.summary()

plot_model(new_model1, show_shapes=True)

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_42 (InputLayer)       [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_20  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_99 (Dense)            (None, 320)               409920    
                                                                 
 batch_normalization_80 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                          

In [101]:
### Chill
# first index is about feature extractor (1: feature extractor 얼리는거 3,4 는 상위층 !)
new_model1.layers[1].set_weights(feature_extractor.layers[1].get_weights())
new_model1.layers[3].set_weights(feature_extractor.layers[3].get_weights())
new_model1.layers[4].set_weights(feature_extractor.layers[4].get_weights())
# 특징추출기에 해당하는 CNN 을 freezing 시킨다.
new_model1.layers[1].trainable = True
new_model1.layers[3].trainable = True
new_model1.layers[4].trainable = True

new_model1.summary()

Epochs = 15

#opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.0, nesterov=False)

#https://keras.io/api/optimizers/learning_rate_schedules/exponential_decay/
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0014,
    decay_steps=Epochs,
    decay_rate=0.5,
    staircase=True)

opt = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule, centered=True)
#opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
#opt = tf.keras.optimizers.Nadam(learning_rate=0.00166, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
new_model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics = ['accuracy'])
my_callbacks1 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/chicago/model_21.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

history1 = new_model1.fit(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=Epochs,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks1])

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_42 (InputLayer)       [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_20  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_99 (Dense)            (None, 320)               409920    
                                                                 
 batch_normalization_80 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                          

In [102]:
history1.history

{'loss': [0.6334965229034424,
  0.5761049389839172,
  0.5739338397979736,
  0.5773021578788757,
  0.5729856491088867,
  0.576383650302887,
  0.5701209306716919,
  0.5717800259590149,
  0.5784355401992798,
  0.575995922088623,
  0.5758357048034668,
  0.5738498568534851,
  0.5780393481254578,
  0.5784282684326172,
  0.5765602588653564],
 'accuracy': [0.8432783484458923,
  0.8680823445320129,
  0.8678483963012695,
  0.8690768480300903,
  0.8690183758735657,
  0.8668538928031921,
  0.8698373436927795,
  0.8710658550262451,
  0.8655083775520325,
  0.8683748841285706,
  0.8669708967208862,
  0.8673803806304932,
  0.8682578802108765,
  0.8664443492889404,
  0.8666783571243286],
 'val_loss': [0.7778360247612,
  0.7592006325721741,
  0.7616700530052185,
  0.7545021176338196,
  0.7484992742538452,
  0.7490745186805725,
  0.7497817277908325,
  0.7569283246994019,
  0.7597276568412781,
  0.7499805688858032,
  0.7514906525611877,
  0.7565299868583679,
  0.7598947286605835,
  0.7581761479377747,
  0

# Bring model and just Train it

In [103]:
feature_extractor = tf.keras.models.load_model('/home/chicago/model_13.h5')

#opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, centered=True)

feature_extractor.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics = ['accuracy'])

In [104]:
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/chicago/model_22.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]
# 128 batch

history = feature_extractor.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=20,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks])

/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/20
134/134 [==============================] - 887s 7s/step - loss: 0.4004 - accuracy: 0.8326 - val_loss: 0.3842 - val_accuracy: 0.8770
Epoch 2/20
134/134 [==============================] - 880s 7s/step - loss: 0.4014 - accuracy: 0.8297 - val_loss: 0.3974 - val_accuracy: 0.8522
Epoch 3/20
134/134 [==============================] - 880s 7s/step - loss: 0.3953 - accuracy: 0.8335 - val_loss: 0.3686 - val_accuracy: 0.8679
Epoch 4/20
134/134 [==============================] - 880s 7s/step - loss: 0.3938 - accuracy: 0.8359 - val_loss: 0.3694 - val_accuracy: 0.8665
Epoch 5/20
134/134 [==============================] - 879s 7s/step - loss: 0.3967 - accuracy: 0.8329 - val_loss: 0.4129 - val_accuracy: 0.8601
Epoch 6/20
134/134 [==============================] - 879s 7s/step - loss: 0.3911 - accuracy: 0.8365 - val_loss: 0.4209 - val_accuracy: 0.8461
Epoch 7/20
134/134 [==============================] - 880s 7s/step - loss: 0.3879 - accuracy: 0.8373 - val_loss: 0.4259 - val_accuracy: 0.8550

In [105]:
history.history

{'loss': [0.40044260025024414,
  0.4014357030391693,
  0.3952963948249817,
  0.3937828540802002,
  0.39669743180274963,
  0.391051322221756,
  0.387897789478302,
  0.38440972566604614,
  0.38537952303886414,
  0.38627710938453674,
  0.382033109664917,
  0.38384127616882324,
  0.37846025824546814,
  0.3791404366493225,
  0.3756340742111206,
  0.37936410307884216,
  0.3777601718902588,
  0.37340009212493896,
  0.3758399188518524,
  0.37313660979270935],
 'accuracy': [0.8325728178024292,
  0.8297063112258911,
  0.8335088491439819,
  0.8359073400497437,
  0.8328653573989868,
  0.8364923596382141,
  0.8372528553009033,
  0.8384228348731995,
  0.8392418622970581,
  0.8362583518028259,
  0.8419328331947327,
  0.8382473587989807,
  0.8415233492851257,
  0.8397098183631897,
  0.8419913649559021,
  0.8395928144454956,
  0.8431028723716736,
  0.8425763249397278,
  0.8425763249397278,
  0.8421668410301208],
 'val_loss': [0.38418635725975037,
  0.3973942995071411,
  0.36858609318733215,
  0.3694210

In [42]:
#https://keras.io/api/optimizers/
#opt = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
#opt = tf.keras.optimizers.Nadam(learning_rate=0.00166, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")

opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])


my_callbacks1 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/media/chicago/Samsung_T5/model/model_1_train.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

'''Before the error
my_callbacks8 = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy', restore_best_weights = True),
    tf.keras.callbacks.ModelCheckpoint(filepath='/media/chicago/Samsung_T5/model/model8')
]
'''


"Before the error\nmy_callbacks8 = [\n    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy', restore_best_weights = True),\n    tf.keras.callbacks.ModelCheckpoint(filepath='/media/chicago/Samsung_T5/model/model8')\n]\n"

In [36]:
history1 = model1.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=epochs,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks1])

/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/8
25/64 [==========>...................] - ETA: 2:54 - loss: 0.6991 - accuracy: 0.7990

KeyboardInterrupt: 

In [ ]:
model1.history.history

In [ ]:
print(model1.evaluate(test_data_gen, return_dict=True))

In [45]:
model2 = keras.models.load_model('/media/chicago/Samsung_T5/Bigdata_Project/model/model_2_train.h5')


#opt = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")

opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, centered=True)
'''
initial_learning_rate = 0.5

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

opt = tf.keras.optimizers.SGD(
    learning_rate=lr_schedule, momentum=0.0, nesterov=False, name="SGD"
)
'''
'''
opt = tf.keras.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    name="Adagrad",
)
'''
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics = ['accuracy'])


my_callbacks2 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/media/chicago/Samsung_T5/model/model_2_train.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

In [46]:
history2 = model2.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=65,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks2])

/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/65
136/136 [==============================] - 339s 2s/step - loss: 0.1739 - accuracy: 0.9321 - val_loss: 0.7506 - val_accuracy: 0.8028
Epoch 2/65
136/136 [==============================] - 326s 2s/step - loss: 0.1332 - accuracy: 0.9528 - val_loss: 0.7299 - val_accuracy: 0.7958
Epoch 3/65
136/136 [==============================] - 325s 2s/step - loss: 0.1157 - accuracy: 0.9555 - val_loss: 0.7640 - val_accuracy: 0.7806
Epoch 4/65
136/136 [==============================] - 325s 2s/step - loss: 0.1184 - accuracy: 0.9577 - val_loss: 0.9233 - val_accuracy: 0.7667
Epoch 5/65
136/136 [==============================] - 332s 2s/step - loss: 0.1015 - accuracy: 0.9643 - val_loss: 1.1968 - val_accuracy: 0.7347
Epoch 6/65
136/136 [==============================] - 326s 2s/step - loss: 0.0922 - accuracy: 0.9643 - val_loss: 1.3262 - val_accuracy: 0.7306
Epoch 7/65
136/136 [==============================] - 329s 2s/step - loss: 0.0921 - accuracy: 0.9668 - val_loss: 0.9671 - val_accuracy: 0.7472

Epoch 58/65
136/136 [==============================] - 329s 2s/step - loss: 0.0276 - accuracy: 0.9907 - val_loss: 2.1431 - val_accuracy: 0.7000
Epoch 59/65
136/136 [==============================] - 326s 2s/step - loss: 0.0287 - accuracy: 0.9914 - val_loss: 2.0076 - val_accuracy: 0.7167
Epoch 60/65
136/136 [==============================] - 327s 2s/step - loss: 0.0264 - accuracy: 0.9914 - val_loss: 2.0349 - val_accuracy: 0.6458
Epoch 61/65
136/136 [==============================] - 326s 2s/step - loss: 0.0194 - accuracy: 0.9939 - val_loss: 3.1812 - val_accuracy: 0.5764
Epoch 62/65
136/136 [==============================] - 331s 2s/step - loss: 0.0301 - accuracy: 0.9907 - val_loss: 2.7908 - val_accuracy: 0.6750
Epoch 63/65
136/136 [==============================] - 329s 2s/step - loss: 0.0273 - accuracy: 0.9902 - val_loss: 1.9487 - val_accuracy: 0.7083
Epoch 64/65
136/136 [==============================] - 327s 2s/step - loss: 0.0210 - accuracy: 0.9926 - val_loss: 2.1219 - val_accuracy:

In [47]:
model2.history.history

{'loss': [0.17387878894805908,
  0.13316020369529724,
  0.11568355560302734,
  0.11841591447591782,
  0.10151355713605881,
  0.0921948179602623,
  0.09209813177585602,
  0.08528725057840347,
  0.0783119648694992,
  0.0813211128115654,
  0.07678598165512085,
  0.060530886054039,
  0.06891383230686188,
  0.06653140485286713,
  0.063510961830616,
  0.06217438727617264,
  0.05805676057934761,
  0.05659937858581543,
  0.05182355269789696,
  0.062227845191955566,
  0.0589875690639019,
  0.04560330510139465,
  0.048309847712516785,
  0.04189646616578102,
  0.04208265617489815,
  0.04527152329683304,
  0.04837792366743088,
  0.04031825438141823,
  0.04803285002708435,
  0.03571443259716034,
  0.042002737522125244,
  0.03608250990509987,
  0.031500015407800674,
  0.044668588787317276,
  0.037420183420181274,
  0.031025273725390434,
  0.032876867800951004,
  0.03231232240796089,
  0.03447895497083664,
  0.03327981382608414,
  0.02790716290473938,
  0.030532849952578545,
  0.035160891711711884,
 

In [48]:
#model saving
model2.save('/media/chicago/Samsung_T5/model/model_2_train_super_overfitting.h5')

In [52]:
model2 = keras.models.load_model('/media/chicago/Samsung_T5/model/model_2_train_super_overfitting.h5')

In [55]:
print(model2.evaluate(val_data_gen, return_dict=True))

25/25 [==============================] - 33s 1s/step - loss: 2.3977 - accuracy: 0.6593
{'loss': 2.3976633548736572, 'accuracy': 0.6592797636985779}


# New Idea

In [ ]:
# 1. after training, set validation set as training set

# 2. set optimizer with SGD and try to escape from local minimum

# 3. change the seed of data and retrain it

In [ ]:
25/25 [==============================] - 34s 1s/step - loss: 2.3706 - accuracy: 0.6537
{'loss': 2.370561122894287, 'accuracy': 0.6537396311759949}

In [127]:
import random

random.seed(42)

In [128]:
print(random.randint(1,10))
print(random.randint(1,10))
print(random.randint(1,10))

2
1
5


In [ ]:
prediction_image_generator = ImageDataGenerator(rescale=1./255)
prediction_data_gen = test_image_generator.flow_from_directory(batch_size=1,
                                                        directory='/media/chicago/Samsung_T5/Bigdata_Project/real_data',
                                                        target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                        class_mode='categorical')

In [ ]:
model2.predict(x_test[:1]

In [ ]:
model4 = keras.models.load_model('/media/chicago/Samsung_T5/Bigdata_Project/model/model_4_train.h5')

#opt = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

model4.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])

my_callbacks4 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/media/chicago/Samsung_T5/model/model_4_train.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]


In [ ]:
history4 = model4.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=epochs,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks4])

In [ ]:
model4.history.history

In [ ]:
print(model4.evaluate(test_data_gen, return_dict=True))

In [ ]:
model5 = keras.models.load_model('/media/chicago/Samsung_T5/Bigdata_Project/model/model_5.h5')

#opt = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)


model5.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])

my_callbacks5 = [
    tf.keras.callbacks.ModelCheckpoint(=
    filepath='/media/chicago/Samsung_T5/model/model_5_train.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]


In [ ]:
history5 = model5.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=epochs,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks5])

In [ ]:
model5.history.history

In [17]:
print(model5.evaluate(test_data_gen, return_dict=True))

265/265 [==============================] - 15s 56ms/step - loss: 1.4041 - accuracy: 0.3396
{'loss': 1.4041026830673218, 'accuracy': 0.3396226465702057}


In [25]:

model7 = keras.models.load_model('/media/chicago/Samsung_T5/Bigdata_Project/model/model_7_train.h5')

#opt = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

model7.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])

my_callbacks7 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/media/chicago/Samsung_T5/model/model_7_train.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]


In [26]:
history7 = model7.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=epochs,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks7])

/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/8
64/64 [==============================] - 353s 5s/step - loss: 0.6796 - accuracy: 0.8775 - val_loss: 1.6307 - val_accuracy: 0.3089
Epoch 2/8
64/64 [==============================] - 348s 5s/step - loss: 0.3783 - accuracy: 0.9316 - val_loss: 1.6118 - val_accuracy: 0.2978
Epoch 3/8
64/64 [==============================] - 346s 5s/step - loss: 0.2997 - accuracy: 0.9479 - val_loss: 1.5840 - val_accuracy: 0.3144
Epoch 4/8
64/64 [==============================] - 349s 5s/step - loss: 0.2615 - accuracy: 0.9631 - val_loss: 1.4824 - val_accuracy: 0.3366
Epoch 5/8
48/64 [=====================>........] - ETA: 1:19 - loss: 0.2409 - accuracy: 0.9655

KeyboardInterrupt: 

In [ ]:
model7.history.history

In [ ]:
print(model7.evaluate(test_data_gen, return_dict=True))

In [ ]:
model8 = keras.models.load_model('/media/chicago/Samsung_T5/Bigdata_Project/model/model_8_train.h5')

#opt = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")

opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

model8.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])

my_callbacks8 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/media/chicago/Samsung_T5/model/model_8_train.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]

In [ ]:
history8 = model8.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=epochs,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    #check call back name
    callbacks=[my_callbacks8])

In [ ]:
model8.history.history

In [ ]:
print(model8
      .evaluate(test_data_gen, return_dict=True))

In [ ]:
#model saving
model2.save('/media/chicago/Samsung_T5/model/model_2_train_decay.h5')

In [58]:
for i, w in enumerate(model8.weights):
    print(i, w.name)

0 Conv1/kernel:0
1 bn_Conv1/gamma:0
2 bn_Conv1/beta:0
3 bn_Conv1/moving_mean:0
4 bn_Conv1/moving_variance:0
5 expanded_conv_depthwise/depthwise_kernel:0
6 expanded_conv_depthwise_BN/gamma:0
7 expanded_conv_depthwise_BN/beta:0
8 expanded_conv_depthwise_BN/moving_mean:0
9 expanded_conv_depthwise_BN/moving_variance:0
10 expanded_conv_project/kernel:0
11 expanded_conv_project_BN/gamma:0
12 expanded_conv_project_BN/beta:0
13 expanded_conv_project_BN/moving_mean:0
14 expanded_conv_project_BN/moving_variance:0
15 block_1_expand/kernel:0
16 block_1_expand_BN/gamma:0
17 block_1_expand_BN/beta:0
18 block_1_expand_BN/moving_mean:0
19 block_1_expand_BN/moving_variance:0
20 block_1_depthwise/depthwise_kernel:0
21 block_1_depthwise_BN/gamma:0
22 block_1_depthwise_BN/beta:0
23 block_1_depthwise_BN/moving_mean:0
24 block_1_depthwise_BN/moving_variance:0
25 block_1_project/kernel:0
26 block_1_project_BN/gamma:0
27 block_1_project_BN/beta:0
28 block_1_project_BN/moving_mean:0
29 block_1_project_BN/movin

# Getting Feature Extractor

In [2]:
model2 = keras.models.load_model('/media/chicago/Samsung_T5/Bigdata_Project/model/model_2_train.h5')

2021-11-21 10:16:53.277183: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 10:16:53.281417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64::/opt/ros/melodic/lib
2021-11-21 10:16:53.282139: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-21 10:16:53.282905: I tensorflow/core/platform/cpu_feature_guard.cc:151] This Ten

In [140]:
feature_extractor = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=.35,
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        pooling=None
      )
    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
feature_extractor.trainable = True

inputs = keras.Input(shape=(320, 320, 3))
x = feature_extractor(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units=256,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=96,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
x = tf.keras.layers.Dense(units=32,activation=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Activation(activation="elu")(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

new_model = keras.Model(inputs=inputs, outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.00166) # MobilenetV2 의 optimal 한 learning rate 를 사용
model2.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), # 교차엔트로피로 구성
          metrics=['accuracy'])

new_model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_3   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 256)               327936    
                                                                 
 batch_normalization_9 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                           

In [148]:
# first index is about feature extractor
new_model.layers[1].set_weights(model2.layers[1].get_weights())

In [149]:
print(model2.layers[1].get_weights())

[array([[[[-1.86854973e-02,  1.01715596e-02, -1.81468248e-01,
          -4.31152587e-21,  2.20785096e-01,  8.19552328e-23,
           9.68125165e-02,  3.78937426e-22, -1.08163513e-01,
          -7.75379464e-02, -6.09713942e-02,  1.61554725e-21,
          -1.40203699e-01,  3.27558474e-22,  2.32023257e-03,
          -7.48481089e-03],
         [ 6.99320063e-02,  5.42734936e-02, -3.06995779e-01,
          -1.03351306e-19,  4.53949660e-01, -3.50681716e-22,
          -4.71903570e-02, -3.65447400e-22, -7.39466846e-02,
           2.87398309e-01,  1.54997304e-01, -1.71896300e-22,
          -2.89402872e-01,  1.77166686e-22, -7.29505643e-02,
           1.68316215e-02],
         [ 2.30636951e-02, -4.27201055e-02, -7.56904110e-02,
          -5.21279076e-20,  1.44612148e-01, -4.06604482e-22,
          -1.04977354e-01, -9.54498787e-23,  9.05383192e-03,
           3.28119099e-02, -1.11675814e-01,  5.23086270e-22,
          -9.30987448e-02, -8.73464596e-23,  1.76529717e-02,
          -2.83944383e-02]],

In [150]:
print(new_model.layers[1].get_weights())

[array([[[[-1.86854973e-02,  1.01715596e-02, -1.81468248e-01,
          -4.31152587e-21,  2.20785096e-01,  8.19552328e-23,
           9.68125165e-02,  3.78937426e-22, -1.08163513e-01,
          -7.75379464e-02, -6.09713942e-02,  1.61554725e-21,
          -1.40203699e-01,  3.27558474e-22,  2.32023257e-03,
          -7.48481089e-03],
         [ 6.99320063e-02,  5.42734936e-02, -3.06995779e-01,
          -1.03351306e-19,  4.53949660e-01, -3.50681716e-22,
          -4.71903570e-02, -3.65447400e-22, -7.39466846e-02,
           2.87398309e-01,  1.54997304e-01, -1.71896300e-22,
          -2.89402872e-01,  1.77166686e-22, -7.29505643e-02,
           1.68316215e-02],
         [ 2.30636951e-02, -4.27201055e-02, -7.56904110e-02,
          -5.21279076e-20,  1.44612148e-01, -4.06604482e-22,
          -1.04977354e-01, -9.54498787e-23,  9.05383192e-03,
           3.28119099e-02, -1.11675814e-01,  5.23086270e-22,
          -9.30987448e-02, -8.73464596e-23,  1.76529717e-02,
          -2.83944383e-02]],

In [152]:
new_model.layers[1].trainable = False

In [153]:
new_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 320, 320, 3)]     0         
                                                                 
 mobilenetv2_0.35_224 (Funct  (None, None, None, 1280)  410208   
 ional)                                                          
                                                                 
 global_average_pooling2d_3   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 256)               327936    
                                                                 
 batch_normalization_9 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                           

In [146]:
for i in range(0,100,1):
    print(len(model2.layers[i].get_weights()))

0
260
0
2
4
0
2
4
0
2
4
0
2


IndexError: list index out of range

In [147]:
for i in range(0,100,1):
    print(len(new_model.layers[i].get_weights()))

0
260
0
2
4
0
2
4
0
2
4
0
2


IndexError: list index out of range

# Hyperparameter tuning

In [84]:
feature_extractor = tf.keras.models.load_model('/media/chicago/Samsung_T5/model/feature_extractor.h5')

In [85]:
#1
def build_model(hp):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=0.35,
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        pooling=None
      )
    base_model.trainable = True
    act = ''
    act_func = hp.Choice(name = "act_func", values = [1,2,3,4], ordered=True)
    if act_func == 1:
        act = "relu"
    elif act_func == 2:
        act = "elu"
    elif act_func == 3:
        act = "selu"
    else:
        act = "softplus"

    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(units=320,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    x = tf.keras.layers.Dense(units=128,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    x = tf.keras.layers.Dense(units=96,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    x = tf.keras.layers.Dense(units=32,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = 3e-4
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model

In [86]:
#2
def build_model(hp):
    alpha = hp.Choice(name = "alpha", values = [0.35,0.5,0.75,1.], ordered=True)
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=alpha,
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        pooling=None
      )
    base_model.trainable = True

    act = 'elu'
    
    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(units=320,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    x = tf.keras.layers.Dense(units=128,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    x = tf.keras.layers.Dense(units=96,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    x = tf.keras.layers.Dense(units=32,activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation=act)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = 3e-4
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model

In [87]:
# Keras_tuner 가 정상 작동하는지 확인
import keras_tuner as kt
build_model(kt.HyperParameters())

2021-11-23 11:00:15.116952: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_119 in the registry.
Traceback (most recent call last):

  File "/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 263, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_119 in the registry.


2021-11-23 11:00:15.117040: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_119 in the registry.
Traceback (most recent call last):

  File "/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 263, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callbac

In [88]:
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    executions_per_trial=1,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [89]:
tuner.search_space_summary()
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


Search space summary
Default search space size: 1
alpha (Choice)
{'default': 0.35, 'conditions': [], 'values': [0.35, 0.5, 0.75, 1.0], 'ordered': True}


In [90]:
tuner.search(train_data_gen,validation_data=val_data_gen, epochs=10, callbacks=[keras.callbacks.TensorBoard("/tmp/alpha_logs")])

Trial 4 Complete [01h 10m 15s]
val_accuracy: 0.7361111044883728

Best val_accuracy So Far: 0.7550504803657532
Total elapsed time: 04h 24m 05s
INFO:tensorflow:Oracle triggered exit


In [93]:
# 텐서보드 출력
%load_ext tensorboard
%tensorboard --logdir /tmp/alpha_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 32109), started 0:10:07 ago. (Use '!kill 32109' to kill it.)

In [97]:
simple_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.04),
                           input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.04)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.04)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128,activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation='relu'),
    tf.keras.layers.Dense(units=64,activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

simple_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 318, 318, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 159, 159, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 157, 157, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 78, 78, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 76, 76, 64)        18496     
                                                                 
 global_average_pooling2d_2   (None, 64)              

In [99]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)

simple_model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])

'''
my_callbacks0 = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='/media/chicago/Samsung_T5/model/model_1_train.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
]


'''

tf_callbacks=[keras.callbacks.TensorBoard("/tmp/alpha_logs")]

In [100]:
history0 = simple_model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//train_batch_size,
    epochs=epochs,
    validation_data = val_data_gen,
    validation_steps = val_num//val_batch_size,
    callbacks=tf_callbacks)

/home/chicago/anaconda3/envs/Big_Data/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
140/140 [==============================] - 330s 2s/step - loss: 2.9018 - accuracy: 0.6898 - val_loss: 2.5714 - val_accuracy: 0.4714
Epoch 2/10
140/140 [==============================] - 326s 2s/step - loss: 1.6949 - accuracy: 0.7695 - val_loss: 2.0073 - val_accuracy: 0.3346
Epoch 3/10
140/140 [==============================] - 325s 2s/step - loss: 1.3095 - accuracy: 0.7776 - val_loss: 1.6923 - val_accuracy: 0.5560
Epoch 4/10
140/140 [==============================] - 326s 2s/step - loss: 1.1204 - accuracy: 0.7863 - val_loss: 1.5774 - val_accuracy: 0.5117
Epoch 5/10
140/140 [==============================] - 327s 2s/step - loss: 1.0160 - accuracy: 0.7805 - val_loss: 1.6716 - val_accuracy: 0.6510
Epoch 6/10
140/140 [==============================] - 327s 2s/step - loss: 0.9253 - accuracy: 0.7915 - val_loss: 1.7273 - val_accuracy: 0.6458
Epoch 7/10
140/140 [==============================] - 330s 2s/step - loss: 0.8520 - accuracy: 0.8020 - val_loss: 1.2662 - val_accuracy: 0.7747

In [4]:
# 텐서보드 출력
%load_ext tensorboard
%tensorboard --logdir /tmp/alpha_logs
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 5303), started 0:00:41 ago. (Use '!kill 5303' to kill it.)

In [ ]:
Brown Color : 0.35
Green Color : 0.5
Pink Color : 0.75
Blue Color : 1
    
    
Brown Color has lowest loss
Blue Color has highest Val accuracy

Thus, I choose 5 models with alpha 1, and the other 5 models I choose alpha 0.35